### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd

In [4]:
##data = pd.read_csv('./prices.csv')
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/AI_ML/Resiency6 Lab/prices.csv')
##!ls "/content/drive/My Drive/AI_ML/Resiency6 Lab/prices.csv"

# New Section

### Check all columns in the dataset

In [6]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop('date',axis=1,inplace=True)

In [0]:
data.drop('symbol',axis=1,inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [10]:
data_reduced = data.iloc[0:1000,:]
data_reduced.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
trainX,testX= train_test_split(data_reduced,test_size=0.3,random_state=100)

In [0]:
trainY = trainX.pop("close")
testY = testX.pop("close")

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x_n,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [19]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:trainX, 
                                                        y_:trainY})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8324.748
Training loss at step:  10  is  6848.473
Training loss at step:  20  is  5858.9277
Training loss at step:  30  is  3883.5466
Training loss at step:  40  is  3883.4219
Training loss at step:  50  is  3883.4104
Training loss at step:  60  is  3883.4028
Training loss at step:  70  is  3883.3975
Training loss at step:  80  is  3883.394
Training loss at step:  90  is  3883.392


In [20]:
sess.run(y, feed_dict={x:trainX[0:1]})

array([[66.62594]], dtype=float32)

In [21]:
testY[0:1]

249    122.169998
Name: close, dtype: float64

In [22]:
sess.run(y, feed_dict={x:testX[0:1]})

array([[66.658165]], dtype=float32)

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [23]:
sess.run(W)

array([[ 0.42802852],
       [-5.0080433 ],
       [ 3.822392  ],
       [ 0.06650945]], dtype=float32)

In [24]:
sess.run(b)

array([26.736284], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [25]:
train_loss = sess.run(loss,feed_dict={x:trainX, 
                                         y_:trainY})
print(train_loss)

3883.3901


In [0]:
test_loss = sess.run(loss,feed_dict={x:testX,y_:testY})

In [27]:
print(test_loss)

2280.5984


In [28]:
print("Absolute mean square loss difference:", abs(train_loss - test_loss))

Absolute mean square loss difference: 1602.7917


### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1))

#Defining the optimizer

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03,clipnorm=1.)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer=sgd_optimizer, loss='mse')

### Execute the model

In [30]:
model.fit(trainX, trainY, epochs=10)

Epoch 1/10
700/700 [==============================] - 0s 327us/step - loss: 7999.3686
Epoch 2/10
700/700 [==============================] - 0s 92us/step - loss: 7492.6544
Epoch 3/10
700/700 [==============================] - 0s 91us/step - loss: 6838.8285
Epoch 4/10
700/700 [==============================] - 0s 94us/step - loss: 6039.7515
Epoch 5/10
700/700 [==============================] - 0s 112us/step - loss: 5138.0973
Epoch 6/10
700/700 [==============================] - 0s 93us/step - loss: 4143.3266
Epoch 7/10
700/700 [==============================] - 0s 95us/step - loss: 3252.9466
Epoch 8/10
700/700 [==============================] - 0s 93us/step - loss: 2339.8238
Epoch 9/10
700/700 [==============================] - 0s 95us/step - loss: 1652.9431
Epoch 10/10
700/700 [==============================] - 0s 88us/step - loss: 1309.2087


### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
tf.set_random_seed(100)

In [0]:
iris = pd.read_csv('/content/drive/My Drive/AI_ML/Resiency6 Lab/Iris.csv')

In [0]:
iris.drop('Id',axis=1,inplace=True)

In [53]:
iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [54]:
iris.Species.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split
trainX_iris,testX_iris= train_test_split(iris,test_size=0.3,random_state=100)
trainY_iris = trainX_iris.pop("Species")
testY_iris = testX_iris.pop("Species")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:

trainY_iris = pd.get_dummies(trainY_iris,drop_first=False)

In [0]:
testY_iris = pd.get_dummies(testY_iris,drop_first=False)

### Divide the dataset into Training and test (70:30)

In [58]:
trainX_iris.shape

(105, 4)

In [59]:
testX_iris.shape

(45, 4)

In [60]:
trainY_iris.shape

(105, 3)

In [61]:
testY_iris

,Iris-setosa,Iris-versicolor,Iris-virginica
128,0,0,1
11,1,0,0
118,0,0,1
15,1,0,0
123,0,0,1
135,0,0,1
32,1,0,0
1,1,0,0
116,0,0,1
45,1,0,0


### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize input data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

In [0]:
#Add First layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))

In [0]:
#Add secon layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))

In [0]:
#Add output layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [68]:
model.fit(trainX_iris,trainY_iris,          
          validation_data=(testX_iris,testY_iris),
          epochs=40,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/40
105/105 [==============================] - 0s 232us/step - loss: 1.0000 - acc: 0.6857 - val_loss: 1.0078 - val_acc: 0.5778
Epoch 2/40
105/105 [==============================] - 0s 189us/step - loss: 0.9982 - acc: 0.6952 - val_loss: 1.0095 - val_acc: 0.5778
Epoch 3/40
105/105 [==============================] - 0s 193us/step - loss: 0.9982 - acc: 0.6190 - val_loss: 1.0069 - val_acc: 0.5778
Epoch 4/40
105/105 [==============================] - 0s 202us/step - loss: 0.9950 - acc: 0.6857 - val_loss: 1.0052 - val_acc: 0.5778
Epoch 5/40
105/105 [==============================] - 0s 193us/step - loss: 0.9911 - acc: 0.6857 - val_loss: 0.9997 - val_acc: 0.5778
Epoch 6/40
105/105 [==============================] - 0s 193us/step - loss: 0.9875 - acc: 0.6952 - val_loss: 0.9985 - val_acc: 0.5778
Epoch 7/40
105/105 [==============================] - 0s 189us/step - loss: 0.9864 - acc: 0.6571 - val_loss: 0.9950 - val_acc: 0.5778
Epoch 8/40
105/10

### Report Accuracy of the predicted values

In [78]:
model.evaluate(testX_iris,testY_iris)

45/45 [==============================] - 0s 180us/step


[0.8579874171151055, 0.8222222248713176]